<a href="https://colab.research.google.com/github/9bishal/Open-Source-Models-with-Hugging-Face/blob/main/Translation_and_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Using the Transformer pipeline

###Importing the important libraries

In [1]:
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from transformers import pipeline

In [3]:
import torch

In [ ]:
translator = pipeline(task="translation",
                      model="facebook/nllb-200-distilled-600M",
                      torch_dtype = torch.bfloat16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

torch_dtype sets the numerical precision (e.g., float16, bfloat16, float32) to optimize speed, memory, and hardware compatibility.

NLLB: No Language Left Behind: 'nllb-200-distilled-600M'.


In [ ]:
text = """\
My puppy is adorable, \
Your kitten is cute.
Her panda is friendly.
His llama is thoughtful. \
We all have nice pets!"""

In [ ]:
text_translated = translator(text,
                             src_lang="eng_Latn",
                             tgt_lang="hin_Deva")

To choose other languages, you can find the other language codes on the page: Languages in FLORES-200

For example:

Afrikaans: afr_Latn

Chinese: zho_Hans

Egyptian Arabic: arz_Arab

French: fra_Latn

German: deu_Latn

Greek: ell_Grek

Hindi: hin_Deva


In [ ]:
text_translated

#2.Direct model inference without using the pipeline abstraction.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TextStreamer

# Load the tokenizer and model
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define source and target languages
src_lang = "fra_Latn"
tgt_lang = "eng_Latn"

# Input sentence
input_text = """
L'intelligence artificielle (IA) connaît une croissance exponentielle au cours des dernières années. Elle est désormais intégrée dans de nombreux aspects de notre vie quotidienne, allant des assistants virtuels comme Siri et Alexa aux systèmes de recommandation sur Netflix, Amazon et YouTube. Dans le domaine de la médecine, l'IA est utilisée pour aider au diagnostic, analyser les images médicales et même proposer des traitements personnalisés aux patients. Dans l'industrie automobile, elle est à la base des voitures autonomes qui pourraient, dans un avenir proche, révolutionner les transports en réduisant les accidents de la route.

L’éducation n’est pas en reste : des plateformes d'apprentissage en ligne utilisent l'IA pour adapter le contenu aux besoins individuels de chaque élève. En agriculture, des systèmes intelligents surveillent les cultures, optimisent l'irrigation et prédisent les récoltes. Cependant, cette évolution rapide soulève également des préoccupations éthiques majeures, notamment en matière de vie privée, de biais algorithmiques et de perte d’emplois due à l’automatisation. Il devient donc essentiel d’encadrer le développement de l’IA par des lois et des politiques appropriées afin de garantir que cette technologie bénéfique soit utilisée de manière équitable, responsable et au service de tous.
"""


# Set the source language for the tokenizer
tokenizer.src_lang = src_lang

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Get the correct token ID for the target language
forced_bos_token_id = tokenizer.convert_tokens_to_ids(f"<<{tgt_lang}>>")

streamer = TextStreamer(tokenizer)

# Generate translation
generated_tokens = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    streamer=streamer
)

# # Decode the generated tokens
# translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0] #use to rmove the special tokens like <um>,</s><unk>

# print("Translated Text:", translation)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TextStreamer

# Load tokenizer and model
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Input and language codes
src_lang = "fra_Latn"
tgt_lang = "eng_Latn"
text = "C'est une belle journée."

# Prepare input
tokenizer.src_lang = src_lang
inputs = tokenizer(text, return_tensors="pt")

# Get target language token ID
forced_bos_token_id = tokenizer.convert_tokens_to_ids(f"<<{tgt_lang}>>")

# Set up streamer to show output while generating
streamer = TextStreamer(tokenizer)

# Generate with streaming
_ = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    streamer=streamer,
    # max_new_tokens=40,
)
